In [ ]:
URL = 'https://www.hepdata.net/record/resource/997020?view=true'
FITTING_SVC='https://us-central1-personalstuff-309012.cloudfunctions.net'
MAX_WORKERS = 1
assert URL is not None
assert FITTING_SVC is not None
assert MAX_WORKERS is not None

import os
os.environ['MPLCONFIGDIR'] = os.environ['HOME']

import requests
import json
import re
import glob
from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import array
from shapely.geometry.polygon import Polygon
from descartes import PolygonPatch

import os
os.environ['MPLCONFIGDIR'] = os.environ['HOME']

import matplotlib.pyplot as plt
from plotting import harvest_results, make_plot, apply_decorations

plt.style.use('atlas')

!rm -rf results
!curl -s {URL} |tar -xzf -
!mkdir -p results

In [ ]:
def func(data):
    filename = data['filename']
    region = data['region']
    m = re.compile("sbottom_(\d+)_(\d+)_(\d+)").search(filename).group(0)
    outname = 'results/region{}.result.{}.json'.format(region,m)
    for i in range(10):
        try:
            d = requests.post(
                '{}/region{}'.format(FITTING_SVC,region),
                data = open(filename), headers = {'Content-Type': 'application/json'}
            ).json()
            json.dump(d,open(outname,'w'))
            break
        except:
            pass#retry

In [ ]:
%matplotlib notebook
fig,ax = plt.subplots(1,1)
fig.set_size_inches(9.33,7)
apply_decorations(ax,label = 'Open Likelihood (in progress)')

In [ ]:
for x in glob.glob('results/*.json'):
    os.unlink(x)
cA = [{'region': 'A', 'filename': f} for f in glob.glob('RegionA/patch*_60.json')]
cC = [{'region': 'C', 'filename': f} for f in glob.glob('RegionC/patch*_60.json')]
configs = cA[:] + cC[:]
# np.random.shuffle(configs)

import time
import concurrent.futures
fig.canvas.draw()

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    for i,_ in enumerate(tqdm(executor.map(func, configs),total = len(configs))):
        if i > 10 and i % 5 == 0:
            make_plot(ax,label = 'Open Likelihood (in progress)', color = 'steelblue', showPoints = True)
            fig.canvas.draw()
        time.sleep(.005)
make_plot(ax, label = 'Open Likelihood', color = 'gold', showPoints = False)


In [ ]:
make_plot(ax, label = 'Open Likelihood', color = 'gold', showPoints = False)